In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# ✅ Define `load_png_slice` function before using it
def load_png_slice(file_path, target_shape=(128, 128)):
    """Load a PNG image as a grayscale slice, resize, and normalize it."""
    try:
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        if img is None:
            raise ValueError(f"Failed to load image: {file_path}")
        
        img = cv2.resize(img, target_shape, interpolation=cv2.INTER_LINEAR)  # Resize
        img = img.astype(np.float32) / 255.0  # Normalize to [0,1]

        return img
    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return None

def get_all_slices(dataset_root):
    """
    Retrieve PNG slices for MCI and NC subjects from Sliced_X_RD, Sliced_Y_RD, and Sliced_Z_RD folders.
    """
    all_slices = []
    all_labels = []
    missing_subjects = []

    categories = ["MCI", "NC"]

    for category in categories:
        category_path = os.path.join(dataset_root, category)
        if not os.path.exists(category_path):
            print(f"⚠️ Warning: Category folder '{category_path}' not found.")
            continue

        for subject in os.listdir(category_path):
            subject_path = os.path.join(category_path, subject)
            if not os.path.isdir(subject_path):
                continue

            # Define paths for required slice folders
            axis_paths = {
                "X": os.path.join(subject_path, "Sliced_X_RD"),
                "Y": os.path.join(subject_path, "Sliced_Y_RD"),
                "Z": os.path.join(subject_path, "Sliced_Z_RD"),
            }

            # Check if all required slice folders exist
            if not all(os.path.exists(path) for path in axis_paths.values()):
                missing_subjects.append(subject)
                continue

            # Collect and sort slice files
            slice_files = {axis: sorted(
                [f for f in os.listdir(axis_paths[axis]) if f.endswith(".png")],
                key=lambda x: int(''.join(filter(str.isdigit, x)))  # Sort numerically
            ) for axis in ["X", "Y", "Z"]}

            # Ensure each axis has slices
            min_slices = min(len(slice_files["X"]), len(slice_files["Y"]), len(slice_files["Z"]))
            if min_slices == 0:
                missing_subjects.append(subject)
                continue

            for i in range(min_slices):
                slice_paths = {
                    "X": os.path.join(axis_paths["X"], slice_files["X"][i]),
                    "Y": os.path.join(axis_paths["Y"], slice_files["Y"][i]),
                    "Z": os.path.join(axis_paths["Z"], slice_files["Z"][i])
                }

                # ✅ Now `load_png_slice` is defined, so this won't cause an error
                slices = [load_png_slice(slice_paths[axis]) for axis in ["X", "Y", "Z"]]

                if any(s is None for s in slices):
                    continue

                combined_slices = np.stack(slices, axis=-1)
                all_slices.append(combined_slices)
                all_labels.append(1 if category == "MCI" else 0)

    if missing_subjects:
        print(f"⚠️ Subjects with missing slices: {len(missing_subjects)}")

    print(f"✅ Total Slices Loaded: {len(all_slices)}")

    return np.array(all_slices, dtype=np.float32), np.array(all_labels, dtype=np.int32)

# 📌 Load Data
dataset_root = r"C:/Users/Alma/Desktop/Data/sliced_data"
data, labels = get_all_slices(dataset_root)

if data.shape[0] == 0:
    print("❌ No valid slices loaded. Please check your dataset paths!")
else:
    print(f"✅ Loaded {len(labels)} samples with shape {data.shape}")  # Shape should be (N, 128, 128, 3)
    print(f"✅ Labels Shape: {labels.shape}")


In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model

# 🔹 Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.set_visible_devices([], 'GPU')
print("✅ Running on CPU only:", tf.config.list_physical_devices('GPU'))

# 🔹 Assume 'data' and 'labels' are numpy arrays already loaded:
# data.shape => (num_samples, 128, 128, 3)
# labels.shape => (num_samples,)

# Function to create ResNet50-based model
def build_resnet_model(input_shape=(128, 128, 3), num_classes=1):
    # 🔹 Input Layer
    inputs = Input(shape=input_shape)

    # 🔹 Base ResNet50 (no pretrained weights)
    base_model = ResNet50(include_top=False, weights=None, input_tensor=inputs)

    # 🔹 Add custom head for binary classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# 🔹 K-Fold Cross Validation Settings
K = 5
EPOCHS = 20
BATCH_SIZE = 16

# StratifiedKFold ensures class distribution consistency
kf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

# Collect fold metrics
fold_accuracies = []
fold_losses = []

# 🔹 K-Fold Loop
for fold, (train_index, val_index) in enumerate(kf.split(data, labels), 1):
    print(f"\n🔹 Fold {fold}/{K} 🔹")

    # Split data
    X_train, X_val = data[train_index], data[val_index]
    y_train, y_val = labels[train_index], labels[val_index]

    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

    # Dataset optimizations
    train_dataset = (train_dataset
                     .shuffle(len(X_train))
                     .batch(BATCH_SIZE)
                     .cache()
                     .prefetch(tf.data.AUTOTUNE))

    val_dataset = (val_dataset
                   .batch(BATCH_SIZE)
                   .cache()
                   .prefetch(tf.data.AUTOTUNE))

    # Build and compile the ResNet50 model
    model = build_resnet_model()

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

    # Train the model
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Evaluate the model on validation data
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)
    print(f"✅ Fold {fold} - Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.4%}")

    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)

# 🔹 Average Metrics Across Folds
print("\n🔹 Cross Validation Results 🔹")
print(f"✅ Average Validation Accuracy: {np.mean(fold_accuracies):.4%} (+/- {np.std(fold_accuracies):.4%})")
print(f"✅ Average Validation Loss: {np.mean(fold_losses):.4f} (+/- {np.std(fold_losses):.4f})")


In [ ]:
#confusion matrix for training dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# ✅ Get Model Predictions for Training Data
y_train_pred_prob = model.predict(train_dataset)  # Get predicted probabilities
y_train_pred = (y_train_pred_prob > 0.5).astype("int32")  # Convert to binary (0 or 1)

# ✅ Convert True Labels to Numpy Array
y_train_true = np.concatenate([y for _, y in train_dataset], axis=0)  # Extract true labels

# ✅ Compute Confusion Matrix for Training Data
cm_train = confusion_matrix(y_train_true, y_train_pred)

# ✅ Compute Accuracy from Confusion Matrix
accuracy_cm_train = np.trace(cm_train) / np.sum(cm_train)  # (TP + TN) / Total Samples

# ✅ Compute Accuracy from Model Evaluation on Training Data
loss_train, accuracy_eval_train = model.evaluate(train_dataset, verbose=0)

# ✅ Plot Training Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', xticklabels=['NC', 'MCI'], yticklabels=['NC', 'MCI'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title(f"Training Confusion Matrix\nModel Accuracy: {accuracy_eval_train:.2%}")
plt.show()

# ✅ Print Accuracy & Classification Report for Training Data
print(f"✅ Training Model Accuracy (from Confusion Matrix): {accuracy_cm_train:.2%}")
print(f"✅ Training Model Accuracy (from Evaluation): {accuracy_eval_train:.2%}")
print("\nTraining Classification Report:\n", classification_report(y_train_true, y_train_pred, target_names=['NC', 'MCI']))


In [ ]:
#confusion matrix for testing dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# ✅ Get Model Predictions
y_pred_prob = model.predict(test_dataset)  # Get predicted probabilities
y_pred = (y_pred_prob > 0.5).astype("int32")  # Convert to binary (0 or 1)

# ✅ Convert True Labels to Numpy Array
y_true = np.concatenate([y for _, y in test_dataset], axis=0)  # Extract true labels

# ✅ Compute Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# ✅ Compute Accuracy from Confusion Matrix
accuracy_cm = np.trace(cm) / np.sum(cm)  # (TP + TN) / Total Samples

# ✅ Compute Accuracy from Model Evaluation
loss, accuracy_eval = model.evaluate(test_dataset, verbose=0)

# ✅ Plot Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NC', 'MCI'], yticklabels=['NC', 'MCI'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title(f"Confusion Matrix\nModel Accuracy: {accuracy_eval:.2%}")
plt.show()

# ✅ Print Accuracy & Classification Report
print(f"✅ Model Accuracy (from Confusion Matrix): {accuracy_cm:.2%}")
print(f"✅ Model Accuracy (from Evaluation): {accuracy_eval:.2%}")
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=['NC', 'MCI']))